In [8]:
import pylast
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import mean_squared_error, r2_score

sys.path.append(os.path.abspath('..'))
from music_sentiment.lastfm_api import get_lastfm_network, get_song_tags
from music_sentiment.tag_utils import process_tag_weights

#load the environment variables from the .env file
load_dotenv()


True

In [9]:
# load the cleaned dataset

df = pd.read_csv("../data/cleaned/muse_cleaned.csv")


In [10]:
# create network for last.fm
network = get_lastfm_network()

# start with a smaller subset of the dataset
sample_size = 2000
df_sample = df.sample(sample_size, random_state=42)

# Fetch tags for each song in the sample
print("Fetching tags for each song in the sample...")
tags_text_list = []
tags_with_weights = []

for idx, row in df_sample.iterrows():
    print(f"Processing {idx+1}/{len(df_sample)}: {row['artist']} - {row['track']}")
    song_tags = get_song_tags(network, row['artist'], row['track'])
    
    # store tag data with weight for each tag
    tags_with_weights.append(song_tags)
    tags_text_list.append(process_tag_weights(song_tags))
    
    
# Add the tags to the DataFrame
df_sample['tags_with_weights'] = tags_with_weights
df_sample['tag_text'] = tags_text_list
# Save the DataFrame with tags to a CSV file
df_sample.to_csv("../data/cleaned/muse_with_tags_valence.csv", index=False)

Fetching tags for each song in the sample...
Processing 53009/2000: Blackstreet - Falling In Love Again
Processing 12457/2000: Lucifers Crossing - Blasphemer
Processing 53406/2000: Madonna - Justify My Love (porno mix) Rare
Processing 64177/2000: Aimee Mann - One
Processing 66341/2000: Linkin Park - Roads Untraveled
Processing 1967/2000: Nits - Two Skaters
Processing 66701/2000: Frank Sinatra - I'll be seeing you
Processing 8342/2000: Frankie Lymon and The Teenagers - Teen Angel
Processing 71666/2000: Vanessa-Mae - City Theme
Processing 6607/2000: Narsilion - Pedraforca, Terra De Bruixes
Processing 23823/2000: T-Bone Walker - Blues For Marili ( LP Version )
Processing 57286/2000: The Harvest Ministers - When You Have A Faint Heart
Processing 55139/2000: Poison - OnceBittenTwiceShy
Processing 39377/2000: The Faint - Desperate Guys
Processing 54512/2000: Beautiful Creatures - 1 A.M.
Processing 35870/2000: Puressence - The Feeling
Processing 29661/2000: Marillion - Beyond You
Processing 2

In [12]:
from sklearn.model_selection import train_test_split, cross_val_score

# Load the dataset with tags
df_sample = pd.read_csv("../data/cleaned/muse_with_tags_valence.csv")

# ComplementNB works with sparse matrices directly

# Your existing code for feature extraction
vectorizer = TfidfVectorizer(min_df=2, max_df=0.7)
X = vectorizer.fit_transform(df_sample['tag_text'])

# Bin the valence scores into discrete categories
bins = [0, 3, 5, 7, 8.5]  # You can adjust these bins
labels = [0, 1, 2, 3]    # Numeric labels for the bins
df_sample['valence_binned'] = pd.cut(df_sample['valence_tags'], bins=bins, labels=labels)

# Use the binned values as target
y = df_sample['valence_binned']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Use ComplementNB which works well with sparse features
model = ComplementNB()
model.fit(X_train, y_train)  # No need for .toarray()

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
from sklearn.metrics import accuracy_score, classification_report
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))



Accuracy: 0.4500
              precision    recall  f1-score   support

           0       0.33      0.33      0.33         6
           1       0.45      0.60      0.51        15
           2       0.60      0.38      0.46        32
           3       0.29      0.57      0.38         7

    accuracy                           0.45        60
   macro avg       0.42      0.47      0.42        60
weighted avg       0.50      0.45      0.45        60



In [14]:
from joblib import dump
import os

# evaluate model on the training data
y_train_pred = model.predict(X_train)

# print evaluation metrics with the same format as the test set
from sklearn.metrics import accuracy_score, classification_report
print(f"Training Data Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(classification_report(y_train, y_train_pred))

# New cell - Save the trained model and vectorizer


# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save the model
dump(model, '../models/valence_classifier.joblib')
# Save the vectorizer
dump(vectorizer, '../models/tfidf_vectorizer.joblib')

print("Model and vectorizer saved successfully!")

Training Data Accuracy: 0.9143
              precision    recall  f1-score   support

           0       1.00      0.62      0.76        13
           1       0.85      0.95      0.90        55
           2       0.94      0.94      0.94        48
           3       1.00      0.96      0.98        24

    accuracy                           0.91       140
   macro avg       0.95      0.86      0.89       140
weighted avg       0.92      0.91      0.91       140

Model and vectorizer saved successfully!
